### Student Information
Name: 劉芷昀

Student ID: 111000206

GitHub ID: Selinaliu1030

Kaggle name: Selina_111000206

Kaggle private scoreboard snapshot: ![alt text](score.png "private_scoreboard")

---

### Instructions

1. First: __This part is worth 30% of your grade.__ Do the **take home exercises** in the [DM2024-Lab2-master Repo](https://github.com/didiersalazar/DM2024-Lab2-Master). You may need to copy some cells from the Lab notebook to this notebook.


2. Second: __This part is worth 30% of your grade.__ Participate in the in-class [Kaggle Competition](https://www.kaggle.com/competitions/dm-2024-isa-5810-lab-2-homework) regarding Emotion Recognition on Twitter by this link: https://www.kaggle.com/competitions/dm-2024-isa-5810-lab-2-homework. The scoring will be given according to your place in the Private Leaderboard ranking:
    - **Bottom 40%**: Get 20% of the 30% available for this section.

    - **Top 41% - 100%**: Get (0.6N + 1 - x) / (0.6N) * 10 + 20 points, where N is the total number of participants, and x is your rank. (ie. If there are 100 participants and you rank 3rd your score will be (0.6 * 100 + 1 - 3) / (0.6 * 100) * 10 + 20 = 29.67% out of 30%.)   
    Submit your last submission **BEFORE the deadline (Nov. 26th, 11:59 pm, Tuesday)**. Make sure to take a screenshot of your position at the end of the competition and store it as '''pic0.png''' under the **img** folder of this repository and rerun the cell **Student Information**.
    

3. Third: __This part is worth 30% of your grade.__ A report of your work developing the model for the competition (You can use code and comment on it). This report should include what your preprocessing steps, the feature engineering steps and an explanation of your model. You can also mention different things you tried and insights you gained.


4. Fourth: __This part is worth 10% of your grade.__ It's hard for us to follow if your code is messy :'(, so please **tidy up your notebook**.


Upload your files to your repository then submit the link to it on the corresponding e-learn assignment.

Make sure to commit and save your changes to your repository __BEFORE the deadline (Nov. 26th, 11:59 pm, Tuesday)__.

###Preparation

In [1]:
### Begin Assignment Here

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip3 install scikit-learn --upgrade
!pip3 install pandas --upgrade
!pip3 install numpy --upgrade
!pip3 install matplotlib --upgrade
!pip3 install plotly --upgrade
!pip3 install seaborn --upgrade
!pip3 install nltk --upgrade
!pip3 install umap-learn --upgrade
!pip3 install gensim --upgrade
!pip3 install tensorflow --upgrade
!pip3 install keras --upgrade
!pip3 install ollama --upgrade
!pip3 install langchain --upgrade
!pip3 install langchain_community --upgrade
!pip3 install langchain_core --upgrade
!pip3 install beautifulsoup4 --upgrade
!pip3 install chromadb --upgrade
!pip3 install gradio --upgrade

In [4]:
# import library
import pandas as pd
import numpy as np
import nltk
import matplotlib.pyplot as plt
import seaborn as sns
import itertools
import umap
import gensim
import tensorflow
import keras
import ollama
import langchain
import langchain_community
import langchain_core
import bs4
import chromadb
import gradio

###Load Data

In [5]:
import pandas as pd
emotion = pd.read_csv("/content/drive/MyDrive/DM/lab2/emotion.csv")
data_id = pd.read_csv("/content/drive/MyDrive/DM/lab2/data_identification.csv")

In [6]:
X = pd.read_json("/content/drive/MyDrive/DM/lab2/tweets_DM.json", lines=True)

X_normalized = pd.json_normalize(X['_source'])
X_normalized.rename(columns={
    'tweet.hashtags': 'hashtags',
    'tweet.tweet_id': 'tweet_id',
    'tweet.text': 'text'
}, inplace=True)

X = pd.concat([X.drop(columns=['_source']), X_normalized], axis=1)
X.head()

,_score,_index,_crawldate,_type,hashtags,tweet_id,text
0,391,hashtag_tweets,2015-05-23 11:42:47,tweets,[Snapchat],0x376b20,"People who post ""add me on #Snapchat"" must be ..."
1,433,hashtag_tweets,2016-01-28 04:52:09,tweets,"[freepress, TrumpLegacy, CNN]",0x2d5350,"@brianklaas As we see, Trump is dangerous to #..."
2,232,hashtag_tweets,2017-12-25 04:39:20,tweets,[bibleverse],0x28b412,"Confident of your obedience, I write to you, k..."
3,376,hashtag_tweets,2016-01-24 23:53:05,tweets,[],0x1cd5b0,Now ISSA is stalking Tasha 😂😂😂 <LH>
4,989,hashtag_tweets,2016-01-08 17:18:59,tweets,[],0x2de201,"""Trust is not the same as faith. A friend is s..."


In [7]:
# Merge X_combined with emotion on tweet_id
X = pd.merge(X, emotion, on='tweet_id', how='left')

# Merge X_combined with data_id on tweet_id
X= pd.merge(X, data_id, on='tweet_id', how='left')

# Split data based on data_id (e.g., 'train' and 'test')
X_train = X[X['identification'] == 'train']
X_test = X[X['identification'] == 'test']

X_train.drop(columns=['identification'], inplace=True)
X_test.drop(columns=['identification'], inplace=True)


<ipython-input-7-d8e44cc00f6c>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train.drop(columns=['identification'], inplace=True)
<ipython-input-7-d8e44cc00f6c>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test.drop(columns=['identification'], inplace=True)


In [8]:
## save to pickle file
X_train.to_pickle("train_df.pkl")
X_test.to_pickle("test_df.pkl")

import pandas as pd

## load a pickle file
train_df = pd.read_pickle("train_df.pkl")
test_df = pd.read_pickle("test_df.pkl")

###Feature Engineering

In [9]:
from sklearn.feature_extraction.text import CountVectorizer

In [10]:
# build analyzers (bag-of-words)
BOW_vectorizer = CountVectorizer()

In [11]:
# 1. Learn a vocabulary dictionary of all tokens in the raw documents.
BOW_vectorizer.fit(train_df['text'])

# 2. Transform documents to document-term matrix.
train_data_BOW_features = BOW_vectorizer.transform(train_df['text'])
test_data_BOW_features = BOW_vectorizer.transform(test_df['text'])

# check the result
train_data_BOW_features

<1455563x794247 sparse matrix of type '<class 'numpy.int64'>'
	with 18849355 stored elements in Compressed Sparse Row format>

In [12]:
# Answer here
import nltk
nltk.download('punkt_tab')
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(max_features=500, tokenizer=nltk.word_tokenize)

# apply analyzer to training data
tfidf.fit(train_df['text'])

train_tfidf = tfidf.transform(train_df['text'])

feature_names_tfidf = tfidf.get_feature_names_out()
feature_names_tfidf[100:110]

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


array(['can', 'car', 'care', 'change', 'christ', 'christmas', 'closed',
       'come', 'comes', 'coming'], dtype=object)

In [13]:
from sklearn.tree import DecisionTreeClassifier

# for a classificaiton problem, you need to provide both training & testing data
X_train = BOW_500.transform(train_df['text'])
y_train = train_df['emotion']

X_test = BOW_500.transform(test_df['text'])
y_test = test_df['emotion']

## take a look at data dimension is a good habit  :)
print('X_train.shape: ', X_train.shape)
print('y_train.shape: ', y_train.shape)
print('X_test.shape: ', X_test.shape)
print('y_test.shape: ', y_test.shape)

NameError: name 'BOW_500' is not defined

### Decision Tree

In [ ]:
## build DecisionTree model
DT_model = DecisionTreeClassifier(random_state=1)

## training!
DT_model = DT_model.fit(X_train, y_train)

## predict!
y_train_pred = DT_model.predict(X_train)
y_test_pred = DT_model.predict(X_test)

## so we get the pred result
y_test_pred[:10]

In [ ]:
## check library
import gensim

## ignore warnings
import warnings
warnings.filterwarnings('ignore')

# # if you want to see the training messages, you can use it
# import logging
# logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

## the input type
train_df['text_tokenized'] = train_df['text'].apply(lambda x: nltk.word_tokenize(x))

## create the training corpus
training_corpus = train_df['text_tokenized'].values


In [ ]:
train_df.head()

In [ ]:
from gensim.models import Word2Vec

## setting
vector_dim = 50
window_size = 5
min_count = 1
training_epochs = 20

## model
word2vec_model = Word2Vec(sentences=training_corpus,
                          vector_size=vector_dim, window=window_size,
                          min_count=min_count, epochs=training_epochs)

In [ ]:
from gensim.models import KeyedVectors

## Note: this model is very huge, this will take some time ...
model_path = "/content/drive/MyDrive/DM/DM2024-Lab2-Master-main/GoogleNews/GoogleNews-vectors-negative300.bin"
w2v_google_model = KeyedVectors.load_word2vec_format(model_path, binary=True)
print('load ok')

w2v_google_model.most_similar('happy', topn=10)

In [ ]:
vector = []
for sentence in training_corpus:
  sum = 0
  for word in sentence:
    if word in w2v_google_model:
      sum += w2v_google_model[word]
  vector.append(sum/len(sentence))

# Apply the function to all tweets
vector_dim = word2vec_model.vector_size  # Dimension of Word2Vec vectors
train_df['text_vector'] = vector

# Convert to a NumPy array for training
X = np.array(train_df['text_vector'].tolist())
y = train_df['emotion'].values

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

# Train the classifier
clf = RandomForestClassifier(random_state=42)
clf.fit(X_train, y_train)

# Predict on the test set
y_test_pred = clf.predict(X_test)

# Evaluate the classifier
print("Accuracy:", accuracy_score(y_test, y_test_pred))
print("\nClassification Report:\n", classification_report(y_test, y_test_pred))


##CNN

In [14]:
import keras

# standardize name (X, y)
X_train = train_tfidf
y_train = train_df['emotion']

X_test = tfidf.transform(test_df['text'])

In [15]:
from sklearn.model_selection import train_test_split
X = train_tfidf  # Your feature matrix (TF-IDF transformed)
y = train_df['emotion']  # Corresponding labels

X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y  # Use stratify to maintain class distribution
)

In [16]:
print('X_train.shape: ', X_train.shape)
print('y_train.shape: ', y_train.shape)
print('X_test.shape: ', X_val.shape)
print('y_test.shape: ', y_val.shape)

X_train.shape:  (1018894, 500)
y_train.shape:  (1018894,)
X_test.shape:  (436669, 500)
y_test.shape:  (436669,)


In [17]:
## deal with label (string -> one-hot)

from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
label_encoder.fit(y_train)
print('check label: ', label_encoder.classes_)
print('\n## Before convert')
print('y_train[0:4]:\n', y_train[0:8])
print('\ny_train.shape: ', y_train.shape)
print('\ny_val.shape: ', y_val.shape)

def label_encode(le, labels):
    enc = le.transform(labels)
    return keras.utils.to_categorical(enc)

def label_decode(le, one_hot_label):
    dec = np.argmax(one_hot_label, axis=1)
    return le.inverse_transform(dec)

y_train = label_encode(label_encoder, y_train)
y_val = label_encode(label_encoder, y_val)

print('\n\n## After convert')
print('y_train[0:4]:\n', y_train[0:8])
print('\ny_train.shape: ', y_train.shape)
print('\ny_val.shape: ', y_val.shape)

check label:  ['anger' 'anticipation' 'disgust' 'fear' 'joy' 'sadness' 'surprise'
 'trust']

## Before convert
y_train[0:4]:
 262074              joy
1707873         sadness
1114819         disgust
1466945             joy
771008     anticipation
26366              fear
1811450             joy
1457182    anticipation
Name: emotion, dtype: object

y_train.shape:  (1018894,)

y_val.shape:  (436669,)


## After convert
y_train[0:4]:
 [[0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0.]]

y_train.shape:  (1018894, 8)

y_val.shape:  (436669, 8)


In [18]:
# I/O check
input_shape = X_train.shape[1]
print('input_shape: ', input_shape)

output_shape = len(label_encoder.classes_)
print('output_shape: ', output_shape)

input_shape:  500
output_shape:  8


In [30]:
from tensorflow import keras
from keras.models import Model
from keras.layers import Input, Dense
from keras.layers import ReLU, Softmax

# input layer
model_input = Input(shape=(input_shape, ))  # 500
X = model_input

# 1st hidden layer
X_W1 = Dense(units=256)(X)  # 64
H1 = ReLU()(X_W1)

# 2nd hidden layer
H1_W2 = Dense(units=128)(H1)  # 64
H2 = ReLU()(H1_W2)

H2_W3 = Dense(units=64)(H2)  # 64
H3 = ReLU()(H2_W3)

# 2nd hidden layer
H3_W4 = Dense(units=32)(H3)  # 64
H4 = ReLU()(H3_W4)

# output layer
H4_W5 = Dense(units=output_shape)(H4)  # 4
H5 = Softmax()(H4_W5)

model_output = H5

# create model
model = Model(inputs=[model_input], outputs=[model_output])

# loss function & optimizer
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# show model construction
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)           │ (None, 500)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 256)                 │         128,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ re_lu_4 (ReLU)                       │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_6 (Dense)                      │ (None, 128)                 │          32,896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ re_lu_5 (ReLU)                       │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_7 (Dense)                      │ (None, 64)                  │           8,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ re_lu_6 (ReLU)                       │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_8 (Dense)                      │ (None, 32)                  │           2,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ re_lu_7 (ReLU)                       │ (None, 32)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_9 (Dense)                      │ (None, 8)                   │             264 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ softmax_1 (Softmax)                  │ (None, 8)                   │               0 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 171,752 (670.91 KB)

 Trainable params: 171,752 (670.91 KB)

 Non-trainable params: 0 (0.00 B)

In [20]:
%matplotlib inline
print("tensorflow: " + tensorflow.__version__)
print("keras: " + keras.__version__)

tensorflow: 2.18.0
keras: 3.7.0


In [21]:
X_train.shape

(1018894, 500)

In [42]:
from keras.callbacks import CSVLogger

csv_logger = CSVLogger('/content/drive/MyDrive/DM/DM2024-Lab2-Master-main/logs/training_log.csv')

# training setting
epochs = 10
batch_size = 32

# training!
history = model.fit(X_train, y_train,
                    epochs=epochs,
                    batch_size=batch_size,
                    callbacks=[csv_logger],
                    validation_data = (X_val, y_val))
print('training finish')

Epoch 1/10
31841/31841 ━━━━━━━━━━━━━━━━━━━━ 309s 10ms/step - accuracy: 0.5672 - loss: 1.1961 - val_accuracy: 0.4896 - val_loss: 1.4456
Epoch 2/10
31841/31841 ━━━━━━━━━━━━━━━━━━━━ 355s 11ms/step - accuracy: 0.5678 - loss: 1.1932 - val_accuracy: 0.4917 - val_loss: 1.4499
Epoch 3/10
31841/31841 ━━━━━━━━━━━━━━━━━━━━ 306s 10ms/step - accuracy: 0.5703 - loss: 1.1893 - val_accuracy: 0.4907 - val_loss: 1.4479
Epoch 4/10
31841/31841 ━━━━━━━━━━━━━━━━━━━━ 349s 11ms/step - accuracy: 0.5711 - loss: 1.1865 - val_accuracy: 0.4907 - val_loss: 1.4661
Epoch 5/10
31841/31841 ━━━━━━━━━━━━━━━━━━━━ 310s 10ms/step - accuracy: 0.5716 - loss: 1.1860 - val_accuracy: 0.4871 - val_loss: 1.4623
Epoch 6/10
31841/31841 ━━━━━━━━━━━━━━━━━━━━ 306s 10ms/step - accuracy: 0.5730 - loss: 1.1827 - val_accuracy: 0.4868 - val_loss: 1.4617
Epoch 7/10
31841/31841 ━━━━━━━━━━━━━━━━━━━━ 324s 10ms/step - accuracy: 0.5739 - loss: 1.1809 - val_accuracy: 0.4881 - val_loss: 1.4692
Epoch 8/10
31841/31841 ━━━━━━━━━━━━━━━━━━━━ 323s 10ms/s

In [43]:
## predict
pred_result = model.predict(X_test, batch_size=128)
pred_result[:5]

3219/3219 ━━━━━━━━━━━━━━━━━━━━ 16s 5ms/step


array([[1.76330318e-03, 7.04985738e-01, 5.44929702e-04, 5.46532730e-03,
        2.16745868e-01, 6.03653537e-03, 1.77823310e-03, 6.26800805e-02],
       [4.70269471e-03, 6.93935335e-01, 4.27669147e-04, 3.42100076e-02,
        1.30682647e-01, 7.27237947e-03, 2.80124648e-03, 1.25967890e-01],
       [1.10607475e-01, 7.56475553e-02, 1.62783831e-01, 9.27365944e-03,
        5.56853414e-02, 5.27598381e-01, 4.48024049e-02, 1.36013906e-02],
       [1.09919417e-03, 4.18167502e-01, 5.38239628e-03, 2.63190689e-03,
        4.83794034e-01, 1.01788593e-02, 2.44821236e-03, 7.62979314e-02],
       [1.18638505e-03, 7.79506087e-01, 9.41287901e-04, 4.00311220e-03,
        1.22946151e-01, 7.83954188e-03, 9.01956111e-04, 8.26754346e-02]],
      dtype=float32)

In [44]:
pred_result = label_decode(label_encoder, pred_result)
pred_result[:5]

array(['anticipation', 'anticipation', 'sadness', 'joy', 'anticipation'],
      dtype=object)

##Cluster

In [ ]:
from sklearn.cluster import KMeans

# Define number of clusters
num_clusters = len(train_df['emotion'].unique())  # Choose based on your data or labels

# Apply K-means clustering
kmeans = KMeans(n_clusters=num_clusters, random_state=42)
kmeans.fit(train_tfidf)


# Predict clusters
X_train['kmeans_cluster'] = kmeans.predict(train_tfidf)

In [ ]:
# Output cluster assignments
print(X_train[['emotion', 'kmeans_cluster']].head())

In [ ]:
from sklearn.metrics import pairwise_distances_argmin_min
train_df = train_df.reset_index(drop=True)
emotion_centroids = {}
for emotion in train_df['emotion'].unique():
    emotion_indices = train_df[train_df['emotion'] == emotion].index
    emotion_centroids[emotion] = np.asarray(train_tfidf[emotion_indices].mean(axis=0)).flatten()

# Map each cluster to the closest emotion centroid
cluster_to_emotion = {}
emotion_centroids_array = np.vstack(list(emotion_centroids.values()))  # Convert centroids to array
for cluster_idx, centroid in enumerate(kmeans.cluster_centers_):
    closest_emotion, _ = pairwise_distances_argmin_min([centroid], emotion_centroids_array)
    cluster_to_emotion[cluster_idx] = list(emotion_centroids.keys())[closest_emotion[0]]


In [ ]:
X_test_tfidf = tfidf.transform(test_df['text'])
# Predict clusters for the test data
y_test_pred_clusters = kmeans.predict(X_test_tfidf)

# Convert cluster predictions to emotion strings
y_test_pred = [cluster_to_emotion[cluster] for cluster in y_test_pred_clusters]

##Save Result

In [45]:
submission = pd.DataFrame({
    'id': np.array(test_df['tweet_id']),
    'emotion': pred_result
})
submission.head()
submission.to_csv('submission.csv', index=False)